# Snow Elastic Modulus Calculations - Bergfeld et al. (2023) Method

This notebook demonstrates the application of the Bergfeld et al. (2023) elastic modulus parameterization to snowpit data. The Bergfeld method calculates elastic modulus from snow density using a power-law relationship optimized from Propagation Saw Test (PST) data.

The analysis uses the local snowpyt_mechparams package and snowpylot for CAAML parsing.


In [1]:
# Import Libraries
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form, parse_sample_pits
from snowpyt_mechparams import density, elastic_modulus


Parse Snowpit Files


In [2]:
# Parse all snowpit files from the data folder
all_pits = parse_sample_pits('data')


Successfully parsed 50278 files
Failed to parse 0 files


Apply Bergfeld parameterization using density and associated uncertainty from:
1. Direct Measurement
2. Kim_geldsetzer

In [ ]:
# CORRECTED: Collect relevant data from each snowpit
pit_info = []
layer_info = []

for pit in all_pits:
    pit_dict = {
        'pit_id': pit.core_info.pit_id,
        'layer_count': len(pit.snow_profile.layers),
    }
    pit_info.append(pit_dict)

    for layer in pit.snow_profile.layers:
        # Create base layer dictionary
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'hand_hardness': layer.hardness,
        }

        # Add kim_geldsetzer grain form conversion if grain form data exists
        if layer.grain_form_primary:
            layer_dict['kim_geldsetzer_grain_form'] = convert_grain_form(layer.grain_form_primary, 'kim_geldsetzer')
            try: 
                # Calculate density using kim_geldsetzer method - only if we have valid inputs
                if layer.hardness and layer_dict['kim_geldsetzer_grain_form']:
                    density_ufloat = density.calculate_density( 
                        method='kim_geldsetzer',
                        hand_hardness=layer.hardness,
                        grain_form=layer_dict['kim_geldsetzer_grain_form']
                    )
                    layer_dict['density_kim_geldsetzer'] = density_ufloat.nominal_value
                    layer_dict['density_kim_geldsetzer_uncertainty'] = density_ufloat.std_dev
                else:
                    layer_dict['density_kim_geldsetzer'] = None
                    layer_dict['density_kim_geldsetzer_uncertainty'] = None
            except Exception:
                layer_dict['density_kim_geldsetzer'] = None
                layer_dict['density_kim_geldsetzer_uncertainty'] = None
        else:
            layer_dict['kim_geldsetzer_grain_form'] = None
            layer_dict['density_kim_geldsetzer'] = None
            layer_dict['density_kim_geldsetzer_uncertainty'] = None

        # Initialize density_measured as None for each layer
        layer_dict['density_measured'] = None
        layer_info.append(layer_dict)

    # Match density observations to layers after all layer_dicts are created
    # Create a mapping of layers by their position for this pit
    pit_layers = [ld for ld in layer_info if ld['pit_id'] == pit.core_info.pit_id]
    pit_layer_start_idx = len(layer_info) - len(pit_layers)
    
    for density_obs in pit.snow_profile.density_profile:
        for i, layer in enumerate(pit.snow_profile.layers):
            if density_obs.depth_top == layer.depth_top and density_obs.thickness == layer.thickness:
                # Found matching layer - update the corresponding layer_dict
                layer_idx = pit_layer_start_idx + i
                if layer_idx < len(layer_info):
                    # Extract scalar value from density
                    density_value = density_obs.density
                    if hasattr(density_value, '__len__') and len(density_value) > 0:
                        density_value = density_value[0]  # Take first element if it's an array
                    if hasattr(density_value, 'nominal_value'):
                        density_value = density_value.nominal_value  # Extract nominal value if it's a ufloat
                    layer_info[layer_idx]['density_measured'] = float(density_value)
                break

# Create a dataframe from the pit and layer info
pit_df = pd.DataFrame(pit_info)
layer_df = pd.DataFrame(layer_info)

layer_df.to_csv('layer_df.csv', index=False)


Apply Bergfeld Elastic Modulus Parameterization

Now we'll apply the Bergfeld et al. (2023) method to calculate elastic modulus for all layers that have density values.


In [4]:
# Apply Bergfeld Elastic Modulus Parameterization to both density methods
from uncertainties import ufloat
import warnings

# Initialize results storage
results = {
    'kim_geldsetzer': {
        'elastic_modulus_values': [],
        'successful_count': 0
    },
    'measured': {
        'elastic_modulus_values': [],
        'successful_count': 0
    }
}

# Process kim_geldsetzer density method (with uncertainty)
print("Processing kim_geldsetzer density method...")
for idx, row in layer_df.iterrows():
    if pd.notna(row['density_kim_geldsetzer']) and pd.notna(row['density_kim_geldsetzer_uncertainty']):
        # Create ufloat with density and uncertainty
        density_ufloat = ufloat(row['density_kim_geldsetzer'], row['density_kim_geldsetzer_uncertainty'])
        
        # Calculate elastic modulus using Bergfeld method
        try:
            E_modulus = elastic_modulus.calculate_elastic_modulus(method='bergfeld', density=density_ufloat)
            
            # Check if result is valid (not NaN)
            if not np.isnan(E_modulus.nominal_value):
                results['kim_geldsetzer']['elastic_modulus_values'].append(E_modulus)
                results['kim_geldsetzer']['successful_count'] += 1
                
                # Store results back in dataframe
                layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer'] = E_modulus.nominal_value
                layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer_uncertainty'] = E_modulus.std_dev
            else:
                layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer'] = np.nan
                layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer_uncertainty'] = np.nan
                
        except Exception as e:
            print(f"Error calculating elastic modulus for kim_geldsetzer method at index {idx}: {e}")
            layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer'] = np.nan
            layer_df.at[idx, 'elastic_modulus_bergfeld_kim_geldsetzer_uncertainty'] = np.nan

# Process measured density method (without uncertainty)
print("Processing measured density method...")
for idx, row in layer_df.iterrows():
    density_measured = row['density_measured']
    
    # Handle the case where density_measured might be an array or have multiple values
    if density_measured is not None:
        # Convert to scalar if it's an array
        if hasattr(density_measured, '__len__') and not isinstance(density_measured, str):
            if len(density_measured) > 0:
                density_measured = density_measured[0]
            else:
                density_measured = None
        
        # Check if we have a valid scalar value
        if density_measured is not None and pd.notna(density_measured):
            # Create ufloat with density but no uncertainty (std_dev = 0)
            density_ufloat = ufloat(float(density_measured), 0.0)
        
            # Calculate elastic modulus using Bergfeld method
            try:
                E_modulus = elastic_modulus.calculate_elastic_modulus(method='bergfeld', density=density_ufloat)
                
                # Check if result is valid (not NaN)
                if not np.isnan(E_modulus.nominal_value):
                    results['measured']['elastic_modulus_values'].append(E_modulus)
                    results['measured']['successful_count'] += 1
                    
                    # Store results back in dataframe
                    layer_df.at[idx, 'elastic_modulus_bergfeld_measured'] = E_modulus.nominal_value
                    layer_df.at[idx, 'elastic_modulus_bergfeld_measured_uncertainty'] = E_modulus.std_dev
                else:
                    layer_df.at[idx, 'elastic_modulus_bergfeld_measured'] = np.nan
                    layer_df.at[idx, 'elastic_modulus_bergfeld_measured_uncertainty'] = np.nan
                    
            except Exception as e:
                print(f"Error calculating elastic modulus for measured method at index {idx}: {e}")
                layer_df.at[idx, 'elastic_modulus_bergfeld_measured'] = np.nan
                layer_df.at[idx, 'elastic_modulus_bergfeld_measured_uncertainty'] = np.nan

# Print summary statistics
print("\n=== BERGFELD ELASTIC MODULUS CALCULATION RESULTS ===")
print(f"Kim-Geldsetzer method:")
print(f"  - Successful calculations: {results['kim_geldsetzer']['successful_count']}")
print(f"  - Total layers with kim_geldsetzer density: {layer_df['density_kim_geldsetzer'].notna().sum()}")

print(f"\nMeasured density method:")
print(f"  - Successful calculations: {results['measured']['successful_count']}")
print(f"  - Total layers with measured density: {layer_df['density_measured'].notna().sum()}")

# Save updated dataframe
layer_df.to_csv('layer_df.csv', index=False)
print(f"\nUpdated layer data saved to layer_df.csv")


Processing kim_geldsetzer density method...


/Users/marykateconnelly/Desktop/snowpylot/snowpylot-applications/SnowPyt-MechParams/test_env/lib/python3.13/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
/Users/marykateconnelly/Desktop/snowpylot/snowpylot-applications/SnowPyt-MechParams/test_env/lib/python3.13/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")


Processing measured density method...


/Users/marykateconnelly/Desktop/snowpylot/snowpylot-applications/SnowPyt-MechParams/test_env/lib/python3.13/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")



=== BERGFELD ELASTIC MODULUS CALCULATION RESULTS ===
Kim-Geldsetzer method:
  - Successful calculations: 235522
  - Total layers with kim_geldsetzer density: 235522

Measured density method:
  - Successful calculations: 10468
  - Total layers with measured density: 10468

Updated layer data saved to layer_df.csv
